<a href="https://colab.research.google.com/github/Charlee0616/Data-Mining/blob/main/Q%26Cproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import math
from sklearn.model_selection import train_test_split
from statistics import mean, stdev

In [3]:
def Attribute_selection_method(dataset, target, method, classification):
  if(method == "classification"): #method chosen was classification
    features = dataset.columns[dataset.columns != target]
    entropys=[]
    for attribute in features:
      entropy = 0
      overall = len( dataset )
      #vals = dataset[attribute].unique()
      if(classification =="entropy"): #classifcation was chosen and entropy was chosen
        for val in vals:
          if(len(dataset[attribute].unique())<20):
            subset_size = len(dataset[ dataset[attribute] == val ])
            weight = subset_size / overall
            props = dataset[ dataset[attribute] == val ][target].value_counts( normalize=True )
            for p in props.array:
              entropy =  entropy - weight*(p*math.log2(p))
        else: #classification was chose and Gini was chosen
          entropy = 0
          left = dataset[ dataset[attribute] <= val ][ [attribute,target] ]
          props = left[ target ].value_counts( normalize = True )
          weight = len( left ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          right = dataset[ dataset[attribute] > val ][ [attribute,target] ]
          props = right[ target ].value_counts( normalize = True )
          weight = len( right ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          entropys.append(entropy)
      else:
        entropy = 1 - sum([p ** 2 for p in props])
        entropy += weight * entropy
        entropys.append(entropy)
    minEnt=min(entropys)
    ind=entropys.index(minEnt)
    return features[ind]
  else: # method is Naive Bayes Classifier/ regressor
    #separate by class
    separated = {}
    for i in range(len(dataset)):
      vector = dataset[i]
      class_value = vector[-1]
      if (class_value not in separated):
        separated[class_value] = []
      separated[class_value].append(vector)

    #Summarize statistivc
    summaries = {}
    for class_value, instances in separated.items():
      summaries[class_value] = [(mean(col), stdev(col), len(col)) for col in zip(*instances)]

    predictions = []
    #put test into the method
    for row in test:
        total_rows = sum([summaries[label][0][2] for label in summaries])
        probabilities = {}
        for class_value, class_summaries in summaries.items():
          probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
          for i in range(len(class_summaries)):
            mean_val, stdev_val, _ = class_summaries[i]
            exponent = math.exp(-((row[i]-mean_val)**2 / (2 * stdev_val**2 )))
            probabilities[class_value] *=(1 / (math.sqrt(2 * math.pi) * stdev_val)) * exponent
        #determine the best label
        best_label, best_prob = None, -1
        for class_value, probability in probabilities.items():
          if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value

        predictions.append(best_label)
    return(predictions)